# Pilot CFM implementation

### Load example programs


In [ ]:
from example.python.crash import crashme, crashme3
from example.python.triangle import triangle, triangle3
from example.python.count import count

import inspect

program = crashme3
print(inspect.getsource(program))

### Fuzzing configuration


In [ ]:
n = 30000
seed_input_list = ["000abcdefghijkl", "100bad!asdfqwer", "010asdfbad!qwer", "001asdfqwerbad!"]
program(seed_input_list[1])

### Run fuzzer and record input, coverage


In [ ]:
from GreyboxFuzzer import GreyboxFuzzerRecorder
from MutationFuzzer import Mutator, PowerSchedule
from Coverage import FunctionContCovRunner

greybox_recorder = GreyboxFuzzerRecorder(
    seed_input_list, Mutator(), PowerSchedule()
)
greybox_recorder.runs(FunctionContCovRunner(program), trials=n)
record = greybox_recorder.get_record()

print(f"Unique path: {len(record)}\n")
for path, inputs in list(record.items()):
    print(f"{path=}")
    print(f"{inputs=}")
    print()

### Create a graph from coverage

In [ ]:
from graph import Graph

contcov_graph = Graph()
coverage_graph = Graph()
for path, inputs in record.items():
    contcov_graph.accept(path)
    coverage_list = [contcov[1] for contcov in path]
    coverage_graph.accept(coverage_list)
# contcov_graph.print_graph()
coverage_graph.print_graph()

In [ ]:
call_contexts = set()
for path, inputs in record.items():
    for cont_cov in path:
        call_stack, _ = cont_cov
        for call_context in call_stack.split('-'):
            if call_context == '':
                continue
            elif call_context.split(':')[0] in ['run_function', 'coverage']:
                continue
            call_contexts.add(call_context)
print(f"{call_contexts=}")

### [Temp] assign temporary contextmap -- need to be changed to learn this

In [ ]:
context_map = {}
context_map["crashme3:13"] = range(3, 7)
context_map["crashme3:15"] = range(7, 11)
context_map["crashme3:17"] = range(11, 15)

### Model formula

In [ ]:
from ControlFlowModel import ControlFlowModel

CFM = ControlFlowModel(record, contcov_graph, coverage_graph, context_map=context_map)
CFM.identify_branch()
CFM.model_condition()

# Previous

### Identify branching point from coverage data


In [ ]:

from ControlFlowModel import ControlFlowModel

coverage_graph = CoverageGraph()
for cov, inputs in list(record.items()):
    print(f"[Coverage]: {cov}, [Input]: {inputs}")
    coverage_graph.accept(cov)
print("\n[Tree]")
coverage_graph.print_tree()

### Build control flow model


In [ ]:
cfm = ControlFlowModel(cov_graph=coverage_graph, record=record)
cfm.model()
print(cfm)